In [5]:
# sales_analysis.ipynb

"""
SALES DATA ANALYSIS PROJECT
================================
 Python Lead (Data Engineer)
 Python Analyst (KPI Calculator)
"""

# ============================================
#  DATA ENGINEER - DATA CLEANING
# ============================================

# Import necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("ROLE 2: DATA ENGINEER - DATA CLEANING PROCESS")
print("=" * 60)
print("\n")

# -----------------------------------------------------------------
# 1. IMPORT THE DATASET
# -----------------------------------------------------------------
print("1. IMPORTING DATASET...")
print("-" * 40)

# Load the CSV file into a pandas DataFrame
# Note: Ensure the CSV file is in the same directory as this notebook
df = pd.read_csv('Week-2-Sales-Data.csv')

# Display basic information about the dataset
print(f"Dataset imported successfully!")
print(f"Shape of dataset: {df.shape}")  # (rows, columns)
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")
print(f"\nFirst 5 rows of the dataset:")
print(df.head())
print("\n")

# -----------------------------------------------------------------
# 2. INITIAL DATA EXPLORATION
# -----------------------------------------------------------------
print("2. INITIAL DATA EXPLORATION...")
print("-" * 40)

# Display column names and data types
print("Column Information:")
print(df.info())
print("\n")

# Display basic statistics for numerical columns
print("Statistical Summary:")
print(df.describe())
print("\n")

# Display unique values in categorical columns
print("Unique values in categorical columns:")
print(f"Products: {df['Product'].unique()}")
print(f"Regions: {df['Region'].unique()}")
print(f"Sales Representatives: {df['Sales_Rep'].unique()}")
print(f"Number of unique orders: {df['Order_ID'].nunique()}")
print("\n")

# -----------------------------------------------------------------
# 3. CHECK FOR DATA QUALITY ISSUES
# -----------------------------------------------------------------
print("3. CHECKING FOR DATA QUALITY ISSUES...")
print("-" * 40)

# 3.1 Check for missing values
print("3.1 Missing Values Check:")
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_report = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percentage
})
print(missing_report[missing_report['Missing Values'] > 0])
if missing_report[missing_report['Missing Values'] > 0].empty:
    print("✓ No missing values found!")
print("\n")

# 3.2 Check for duplicates
print("3.2 Duplicates Check:")
duplicate_rows = df.duplicated().sum()
duplicate_order_ids = df['Order_ID'].duplicated().sum()
print(f"Total duplicate rows: {duplicate_rows}")
print(f"Duplicate Order IDs: {duplicate_order_ids}")
if duplicate_rows == 0 and duplicate_order_ids == 0:
    print("✓ No duplicates found!")
print("\n")

# 3.3 Check data types
print("3.3 Data Types Check:")
print(df.dtypes)
print("\n")

# 3.4 Check for potential data inconsistencies
print("3.4 Data Consistency Checks:")
print("Checking Revenue calculation consistency...")
# Calculate expected revenue from Units_Sold * Unit_Price
df['Calculated_Revenue'] = df['Units_Sold'] * df['Unit_Price']

# Compare with existing Revenue column
revenue_discrepancies = df[df['Revenue'] != df['Calculated_Revenue']]
print(f"Revenue discrepancies found: {len(revenue_discrepancies)}")
if len(revenue_discrepancies) == 0:
    print("✓ All revenue calculations are consistent!")
else:
    print("\nRevenue discrepancies:")
    print(revenue_discrepancies[['Order_ID', 'Units_Sold', 'Unit_Price', 'Revenue', 'Calculated_Revenue']])
print("\n")

# -----------------------------------------------------------------
# 4. DATA CLEANING PROCESS
# -----------------------------------------------------------------
print("4. PERFORMING DATA CLEANING...")
print("-" * 40)

# Create a copy of the original dataframe for cleaning
df_clean = df.copy()

# 4.1 Remove the temporary calculated column
df_clean = df_clean.drop('Calculated_Revenue', axis=1)
print("4.1 Removed temporary calculated column")
print("\n")

# 4.2 Convert Order_Date to proper datetime format
print("4.2 Converting Order_Date to datetime format...")
df_clean['Order_Date'] = pd.to_datetime(df_clean['Order_Date'])
print(f"✓ Order_Date converted to datetime: {df_clean['Order_Date'].dtype}")
print(f"Date range: {df_clean['Order_Date'].min()} to {df_clean['Order_Date'].max()}")
print("\n")

# 4.3 Check for and handle any missing values (if any were found)
# Since we found no missing values, this is just for demonstration
print("4.3 Missing value handling (demonstration):")
if df_clean.isnull().sum().sum() > 0:
    print("Handling missing values...")
    # For numerical columns, fill with median
    # For categorical columns, fill with mode
    numerical_cols = df_clean.select_dtypes(include=[np.number]).columns
    categorical_cols = df_clean.select_dtypes(include=['object']).columns
    
    for col in numerical_cols:
        if df_clean[col].isnull().sum() > 0:
            df_clean[col].fillna(df_clean[col].median(), inplace=True)
    
    for col in categorical_cols:
        if df_clean[col].isnull().sum() > 0:
            df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)
    print("Missing values handled.")
else:
    print("✓ No missing values to handle!")
print("\n")

# 4.4 Remove duplicates (if any were found)
print("4.4 Duplicate handling (demonstration):")
initial_rows = len(df_clean)
df_clean = df_clean.drop_duplicates()
final_rows = len(df_clean)
duplicates_removed = initial_rows - final_rows

if duplicates_removed > 0:
    print(f"Removed {duplicates_removed} duplicate rows")
else:
    print("✓ No duplicates to remove!")
print("\n")

# 4.5 Validate data types
print("4.5 Final data type validation:")
print(df_clean.dtypes)
print("\n")

# 4.6 Check for outliers in numerical columns
print("4.6 Outlier detection (summary):")
numerical_cols = ['Units_Sold', 'Unit_Price', 'Revenue']
for col in numerical_cols:
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df_clean[(df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)]
    print(f"{col}: {len(outliers)} potential outliers detected")
print("\n")

# -----------------------------------------------------------------
# 5. FINAL CLEANED DATASET SUMMARY
# -----------------------------------------------------------------
print("5. CLEANED DATASET SUMMARY")
print("-" * 40)
print(f"Final shape: {df_clean.shape}")
print(f"\nFirst 3 rows of cleaned dataset:")
print(df_clean.head(3))
print(f"\nData types after cleaning:")
print(df_clean.dtypes)
print("\n" + "=" * 60)
print("DATA CLEANING COMPLETED SUCCESSFULLY!")
print("=" * 60 + "\n\n")

# ============================================
# ROLE 3: PYTHON ANALYST - KPI CALCULATIONS
# ============================================

print("=" * 60)
print("ROLE 3: PYTHON ANALYST - KPI CALCULATIONS")
print("=" * 60)
print("\n")

# -----------------------------------------------------------------
# 1. SETUP FOR ANALYSIS
# -----------------------------------------------------------------
# We'll use the cleaned dataframe for all calculations
df_analysis = df_clean.copy()

# Add month and year columns for time-based analysis
df_analysis['Order_Month'] = df_analysis['Order_Date'].dt.month
df_analysis['Order_Year'] = df_analysis['Order_Date'].dt.year
df_analysis['Month_Name'] = df_analysis['Order_Date'].dt.strftime('%B')

print("Added time-based columns for analysis:")
print(df_analysis[['Order_Date', 'Order_Month', 'Month_Name', 'Order_Year']].head())
print("\n")

# -----------------------------------------------------------------
# 2. CALCULATE KEY PERFORMANCE INDICATORS (KPIs)
# -----------------------------------------------------------------
print("2. CALCULATING KEY PERFORMANCE INDICATORS")
print("-" * 40)
print("\n")

# KPI 1: Total revenue for the entire dataset
print("KPI 1: TOTAL REVENUE")
print("-" * 20)
total_revenue = df_analysis['Revenue'].sum()
print(f"Total Revenue: R {total_revenue:,.2f}")
print(f"Formatted: R {total_revenue:,.2f}")
print("\n")

# KPI 2: Average units sold per order
print("KPI 2: AVERAGE UNITS SOLD PER ORDER")
print("-" * 20)
avg_units_sold = df_analysis['Units_Sold'].mean()
print(f"Average Units Sold per Order: {avg_units_sold:.2f}")
print(f"Median Units Sold per Order: {df_analysis['Units_Sold'].median():.2f}")
print(f"Range: {df_analysis['Units_Sold'].min()} to {df_analysis['Units_Sold'].max()}")
print("\n")

# KPI 3: Total revenue per region
print("KPI 3: TOTAL REVENUE PER REGION")
print("-" * 20)
revenue_by_region = df_analysis.groupby('Region')['Revenue'].sum().sort_values(ascending=False)
print("Revenue by Region (Descending Order):")
for region, revenue in revenue_by_region.items():
    print(f"  {region}: R {revenue:,.2f}")
print("\n")

# Create a formatted table
region_summary = pd.DataFrame({
    'Region': revenue_by_region.index,
    'Total Revenue': revenue_by_region.values,
    'Percentage of Total': (revenue_by_region.values / total_revenue * 100).round(2)
})
print(region_summary.to_string(index=False))
print("\n")

# KPI 4: Highest revenue-generating sales representative
print("KPI 4: HIGHEST REVENUE-GENERATING SALES REPRESENTATIVE")
print("-" * 20)
revenue_by_rep = df_analysis.groupby('Sales_Rep')['Revenue'].sum().sort_values(ascending=False)
print("Top 5 Sales Representatives by Revenue:")
for i, (rep, revenue) in enumerate(revenue_by_rep.head().items(), 1):
    print(f"  {i}. {rep}: R {revenue:,.2f}")

# Get the top rep
top_rep = revenue_by_rep.index[0]
top_rep_revenue = revenue_by_rep.iloc[0]
print(f"\n🏆 Highest Revenue Generating Sales Rep: {top_rep}")
print(f"   Total Revenue Generated: R {top_rep_revenue:,.2f}")
print(f"   Percentage of Total Revenue: {(top_rep_revenue/total_revenue*100):.2f}%")
print("\n")

# KPI 5: Top 3 products by total units sold
print("KPI 5: TOP 3 PRODUCTS BY TOTAL UNITS SOLD")
print("-" * 20)
units_by_product = df_analysis.groupby('Product')['Units_Sold'].sum().sort_values(ascending=False)
print("All Products by Units Sold:")
for product, units in units_by_product.items():
    print(f"  {product}: {units:,} units")

# Get top 3
top_3_products = units_by_product.head(3)
print(f"\n🥇 Top 3 Products by Units Sold:")
for i, (product, units) in enumerate(top_3_products.items(), 1):
    print(f"  {i}. {product}: {units:,} units")
print("\n")

# -----------------------------------------------------------------
# 3. ADDITIONAL INSIGHTS AND ANALYSIS
# -----------------------------------------------------------------
print("3. ADDITIONAL INSIGHTS")
print("-" * 40)
print("\n")

# 3.1 Monthly revenue trend
print("3.1 MONTHLY REVENUE TREND")
monthly_revenue = df_analysis.groupby(['Order_Year', 'Order_Month', 'Month_Name'])['Revenue'].sum().reset_index()
monthly_revenue = monthly_revenue.sort_values(['Order_Year', 'Order_Month'])

# Create a readable month-year format
monthly_revenue['Month_Year'] = monthly_revenue.apply(
    lambda x: f"{x['Month_Name']} {x['Order_Year']}", axis=1
)

print("Monthly Revenue Summary:")
for _, row in monthly_revenue.iterrows():
    print(f"  {row['Month_Year']}: R {row['Revenue']:,.2f}")
print("\n")

# 3.2 Product performance by revenue
print("3.2 PRODUCT PERFORMANCE BY REVENUE")
product_revenue = df_analysis.groupby('Product')['Revenue'].sum().sort_values(ascending=False)
print("Products Ranked by Revenue:")
for i, (product, revenue) in enumerate(product_revenue.items(), 1):
    print(f"  {i}. {product}: R {revenue:,.2f}")
print("\n")

# 3.3 Regional performance analysis
print("3.3 REGIONAL PERFORMANCE DEEP DIVE")
regional_stats = df_analysis.groupby('Region').agg({
    'Revenue': ['sum', 'mean', 'count'],
    'Units_Sold': ['sum', 'mean']
}).round(2)

# Flatten column names
regional_stats.columns = ['_'.join(col).strip() for col in regional_stats.columns.values]
print(regional_stats)
print("\n")

# 3.4 Sales representative performance metrics
print("3.4 SALES REPRESENTATIVE PERFORMANCE METRICS")
rep_performance = df_analysis.groupby('Sales_Rep').agg({
    'Order_ID': 'count',
    'Revenue': ['sum', 'mean'],
    'Units_Sold': ['sum', 'mean']
}).round(2)

rep_performance.columns = ['Orders', 'Total_Revenue', 'Avg_Revenue', 'Total_Units', 'Avg_Units']
rep_performance = rep_performance.sort_values('Total_Revenue', ascending=False)

print("Top 10 Sales Representatives:")
print(rep_performance.head(10))
print("\n")

# -----------------------------------------------------------------
# 4. EXPORT RESULTS
# -----------------------------------------------------------------
print("4. EXPORTING RESULTS")
print("-" * 40)

# Create a summary dataframe for export
summary_data = {
    'KPI': [
        'Total Revenue',
        'Average Units per Order',
        'Top Sales Representative',
        'Top Sales Rep Revenue',
        'Top Product by Units',
        'Top Product Units Sold'
    ],
    'Value': [
        f"R {total_revenue:,.2f}",
        f"{avg_units_sold:.2f}",
        top_rep,
        f"R {top_rep_revenue:,.2f}",
        top_3_products.index[0],
        f"{top_3_products.iloc[0]:,}"
    ]
}

summary_df = pd.DataFrame(summary_data)

# Export to CSV for use in Excel report
summary_df.to_csv('kpi_summary.csv', index=False)
revenue_by_region.to_csv('revenue_by_region.csv')
units_by_product.to_csv('units_by_product.csv')
monthly_revenue.to_csv('monthly_revenue.csv')

print("✓ KPI Summary exported to 'kpi_summary.csv'")
print("✓ Revenue by Region exported to 'revenue_by_region.csv'")
print("✓ Units by Product exported to 'units_by_product.csv'")
print("✓ Monthly Revenue exported to 'monthly_revenue.csv'")
print("\n")

# -----------------------------------------------------------------
# 5. FINAL SUMMARY AND VISUALIZATION PREVIEW
# -----------------------------------------------------------------
print("=" * 60)
print("PROJECT SUMMARY")
print("=" * 60)

print("\n📊 KEY FINDINGS:")
print(f"1. Total Revenue: R {total_revenue:,.2f}")
print(f"2. Average Units per Order: {avg_units_sold:.2f}")
print(f"3. Top Region by Revenue: {revenue_by_region.index[0]} (R {revenue_by_region.iloc[0]:,.2f})")
print(f"4. Top Sales Rep: {top_rep} (R {top_rep_revenue:,.2f})")
print(f"5. Top Product by Units: {top_3_products.index[0]} ({top_3_products.iloc[0]:,} units)")

print("\n📈 MONTHLY TREND:")
for _, row in monthly_revenue.iterrows():
    print(f"  {row['Month_Year']}: R {row['Revenue']:,.2f}")

print("\n" + "=" * 60)
print("ANALYSIS COMPLETED SUCCESSFULLY!")
print("=" * 60)

# Display final dataframe info
print("\n\nFINAL DATASET INFO:")
print(f"Total Records: {len(df_analysis)}")
print(f"Date Range: {df_analysis['Order_Date'].min().date()} to {df_analysis['Order_Date'].max().date()}")
print(f"Unique Products: {df_analysis['Product'].nunique()}")
print(f"Unique Regions: {df_analysis['Region'].nunique()}")
print(f"Unique Sales Reps: {df_analysis['Sales_Rep'].nunique()}")

ROLE 2: DATA ENGINEER - DATA CLEANING PROCESS


1. IMPORTING DATASET...
----------------------------------------


FileNotFoundError: [Errno 2] No such file or directory: 'Week-2-Sales-Data.csv'